<h1> Training flight delay model in BigQuery ML </h1>

Run this notebook in Vertex Workbench. In this notebook, we will use BigQuery ML to train the same model that we did in Spark ML.

Note how much easier this is ... (and also much more scaleable)

## Verify dataset

Let's make sure that we have the traindays and flights data in BigQuery. If you don't, please follow steps in the README.md in this directory.

In [1]:
%%bigquery
SELECT * FROM dsongcp.flights_tzcorr
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.14rows/s]


,FL_DATE,UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_SEQ_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,...,ARR_DELAY,CANCELLED,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET
0,2015-12-09,AS,1029904,ANC,1055102,BET,2015-12-10 04:00:00+00:00,2015-12-10 03:55:00+00:00,-5.0,10.0,...,-3.0,False,False,399.0,61.174167,-149.998056,-32400.0,60.778611,-161.837222,-32400.0
1,2015-02-07,B6,1169703,FLL,1484304,SJU,2015-02-07 13:50:00+00:00,2015-02-07 14:23:00+00:00,33.0,13.0,...,22.0,False,False,1046.0,26.072500,-80.152778,-18000.0,18.439444,-66.002222,-14400.0
2,2015-10-29,AS,1482803,SIT,1252303,JNU,2015-10-29 14:00:00+00:00,2015-10-29 13:57:00+00:00,-3.0,8.0,...,10.0,False,False,95.0,57.046944,-135.361111,-28800.0,58.354722,-134.574722,-28800.0
3,2015-10-02,HA,1298202,LIH,1383002,OGG,2015-10-02 22:53:00+00:00,2015-10-02 22:58:00+00:00,5.0,9.0,...,8.0,False,False,201.0,21.976111,-159.338889,-36000.0,20.898611,-156.430556,-36000.0
4,2015-06-30,AS,1075403,BRW,1470903,SCC,2015-07-01 04:20:00+00:00,2015-07-01 04:16:00+00:00,-4.0,5.0,...,-8.0,False,False,204.0,71.284722,-156.768611,-28800.0,70.195556,-148.465833,-28800.0


In [3]:
%%bigquery
SELECT * FROM dsongcp.trainday
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.52rows/s]


,FL_DATE,is_train_day
0,2015-01-01,True
1,2015-01-02,False
2,2015-01-03,False
3,2015-01-04,True
4,2015-01-05,True


## Logistic regression

Let's use SQL to craft the dataset just the way we want it.

In [4]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.66rows/s]


,ontime,dep_delay,taxi_out,distance,is_eval_day
0,ontime,-5.0,10.0,399.0,True
1,late,33.0,13.0,1046.0,False
2,ontime,-3.0,8.0,95.0,False
3,ontime,5.0,9.0,201.0,True
4,ontime,-4.0,5.0,204.0,False


Train the model:
* ontime column in the label
* the model is a logistic regression
* use the is_eval_day column to split the data
* remaining columns are features

This will take about 10 minutes.

In [12]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1061.58query/s]                        


""


In [13]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 20/20 [00:01<00:00, 18.82rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,19,0.000003,0.000004,104857.6,3306
1,0,18,0.000007,0.000007,52428.8,3350
2,0,17,0.000013,0.000012,26214.4,2941
3,0,16,0.000027,0.000020,13107.2,2921
4,0,15,0.000054,0.000033,6553.6,3370
5,0,14,0.000108,0.000058,3276.8,2693
6,0,13,0.000216,0.000102,1638.4,3356
7,0,12,0.000432,0.000182,819.2,3068
8,0,11,0.000865,0.000333,409.6,3291
9,0,10,0.001734,0.000625,204.8,3051


In [15]:
%%bigquery
SELECT * FROM ML.WEIGHTS(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.72rows/s]


,processed_input,weight,category_weights
0,dep_delay,-0.132984,[]
1,taxi_out,-0.121715,[]
2,distance,0.000223,[]
3,__INTERCEPT__,4.762572,[]


In [31]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                        (
SELECT 12.0 AS dep_delay, 14.0 AS taxi_out, 1231 AS distance
                        ))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.12s/rows]


,predicted_ontime,predicted_ontime_probs,dep_delay,taxi_out,distance
0,ontime,"[{'label': 'ontime', 'prob': 0.850350772376706...",12.0,14.0,1231


In [18]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_lm,
                 (
                     
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
    is_train_day = 'False'
                     
                 ),
                 STRUCT(0.7 AS threshold))

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.39s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.964337,0.956535,0.935174,0.96042,0.167233,0.956248


## Do same metrics as in Spark code

We are using ML.PREDICT and computing the necessary stats

In [22]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.14s/rows]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.836363,1301948,0.964337,283750,0.213091


## Add airport info

Add airport information to model (note two additional columns: origin and dest). This seemingly simple change adds two categorical variables that, when one-hot-encoded, adds 600+ new columns to the model. BigQuery ML doesn't skip a beat ...

This query will take ~10 minutes

In [25]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1256.16query/s]                        


""


In [26]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.11s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.967151,0.957706,0.938477,0.962405,0.165557,0.960821


In [27]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.01rows/s]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.84953,1299749,0.967151,283750,0.209839


Note that the addition of the airports information has improved both the AUC and the RMSE

Copyright 2019-2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.